In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

C:\Users\User\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [4]:
df = pd.read_csv('protine_greay.csv')
df.head()

,0,0.911049312,0.893364201,1.367705578,1.174658728,0.690775628,1.376112978,0.841920089,0.9767566,0.996547468,...,0.870866555,0.798936468,1.097323493,1.241005069,1.158627079,1.172114342,0.953118336,1.354495246,1.000999631,1.000003861
0,0,1.387036,0.698265,1.078488,0.726582,1.223709,1.088100,0.907629,1.267496,0.795251,...,1.007925,0.772375,0.930409,0.893880,0.880563,1.014962,0.898747,0.884111,0.799560,1.007950
1,0,1.013362,1.280073,1.062973,0.663985,1.021497,1.024406,0.878744,1.065177,1.170897,...,1.211608,0.831024,0.787963,0.855939,1.145608,0.824321,1.144456,0.885583,0.795038,1.018730
2,0,0.871261,1.206636,0.942709,0.781360,0.772901,1.111084,1.181474,1.125368,0.617150,...,0.924218,0.939042,1.191293,1.139526,0.818825,1.317939,1.181889,1.057030,1.080960,0.353841
3,0,1.259478,1.060013,0.874230,1.263320,0.860889,0.926173,1.324730,0.763908,0.869476,...,1.212882,1.231901,0.900892,0.900512,1.059143,1.179653,1.022106,0.821552,1.001487,1.350223
4,0,1.063716,1.190811,0.750346,0.803915,0.646880,0.908342,0.659384,0.929710,0.677770,...,0.794670,1.055793,1.319425,0.611117,1.218067,0.974000,1.139521,0.847896,1.007993,1.109879


In [5]:
y = df['0']
x = df.drop('0', axis=1) 

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [7]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.619860  0.011826   
1  XGBClassifier(base_score=None, booster=None, c...  0.588766  0.035099   
2      LGBMClassifier(max_depth=10, random_state=50)  0.564694 -0.012904   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.552658 -0.012911   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.604814 -0.004919   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.007027   0.389831  0.062842  0.108235     0.942948     0.062842  
1  0.032891   0.399083  0.237705  0.297945     0.792393     0.237705  
2 -0.012296   0.355932  0.229508  0.279070     0.759113     0.229508  
3 -0.012662   0.357143  0.273224  0.309598     0.714739     0.273224  
4 -0.003618   0.360000  0.098361  0.154506     0.898574     0.098361  


In [8]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.737718  0.475465   
1  XGBClassifier(base_score=None, booster=None, c...  0.710777  0.421787   
2      LGBMClassifier(max_depth=10, random_state=50)  0.695721  0.391946   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.688590  0.378417   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.581616  0.163351   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.475436   0.740385  0.732171  0.736255     0.743265     0.732171  
1  0.421553   0.703988  0.727417  0.715511     0.694136     0.727417  
2  0.391442   0.686275  0.721078  0.703246     0.670365     0.721078  
3  0.377179   0.674487  0.729002  0.700685     0.648177     0.729002  
4  0.163233   0.578626  0.600634  0.589425     0.562599     0.600634  
